In [1]:
import numpy as np
import pandas as pd
from itertools import product
import os
from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt

from pyproj import CRS, Transformer
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt

### Locate data

In [2]:
# Locate files
source = '/media/Data_storage/Mobilcell/Data/'
#source = '/home/abiricz/Projects/Mobilcell/Data/'

In [3]:
files_poligons = np.array( sorted([ i for i in os.listdir(source) if 'POLIGONS' in i]) )
files_poligons.shape, files_poligons[:3]

((365,), array(['MT_MTA_NETWORK_POLIGONS_20181201.csv.gz',
        'MT_MTA_NETWORK_POLIGONS_20181202.csv.gz',
        'MT_MTA_NETWORK_POLIGONS_20181203.csv.gz'], dtype='<U39'))

### Calculate universal grid for rasters

In [4]:
# This seems to be the global grid of rasters, same for every day!
start_x = 48262 # 48210 + 127 - ( 48210 % 127 )
end_x = 362968 # 366660 + 127 - ( 366660 % 127 )

start_y = 426468 # 426341 + ( 426341 % 127 )
end_y = 934214 # 934219 + ( 934219 % 127 )

num_x = int( ( end_x - start_x ) / 127 )
num_y = int( ( end_y - start_y ) / 127 )

print( start_x, end_x, start_y, end_y, num_x, num_y )

# this raster encoding is universal then
raster_x = np.arange(start_x, end_x+127, 127, dtype=np.int32)
raster_y = np.arange(start_y, end_y+127, 127, dtype=np.int32)

# get coordinate vector
raster_coords = np.array( list(product( raster_x, raster_y )) )

48262 362968 426468 934214 2478 3998


### Load data

In [5]:
poligons_path = source + files_poligons[100]
poligons_df = pd.read_csv( poligons_path, delimiter=';' )

# drop poligons outside of the country
poligons_df = poligons_df[ np.logical_and( poligons_df.eovx.values < 366660, 
                                   poligons_df.eovx.values > 48210 ) ]
poligons_df = poligons_df[ np.logical_and( poligons_df.eovy.values < 934219, 
                                   poligons_df.eovy.values > 426341 ) ]

In [6]:
poligons_df.head()

,network_identifier,eovx,eovy,dx,dy,network_element_type
0,10071347,216918,496318,127,127,D
1,10071347,216918,496445,127,127,D
2,10071347,217045,496191,127,127,D
3,10071347,217045,496318,127,127,D
4,10071347,217045,496572,127,127,D


### Match raster index with the universal grid

In [7]:
# calculate raster encodings
poligons_df['eovx_num'] = ( (poligons_df.eovx - start_x) / 127 ).astype(int)
poligons_df['eovy_num'] = ( (poligons_df.eovy - start_y) / 127 ).astype(int)
poligons_df['eov_idx'] = poligons_df.eovx_num * (num_y+1) + poligons_df.eovy_num

In [8]:
poligons_df.head()

,network_identifier,eovx,eovy,dx,dy,network_element_type,eovx_num,eovy_num,eov_idx
0,10071347,216918,496318,127,127,D,1328,550,5311222
1,10071347,216918,496445,127,127,D,1328,551,5311223
2,10071347,217045,496191,127,127,D,1329,549,5315220
3,10071347,217045,496318,127,127,D,1329,550,5315221
4,10071347,217045,496572,127,127,D,1329,552,5315223


In [9]:
#unique_all = []
towers_types = []
for i in tqdm( range(files_poligons.shape[0]) ):
    # load data
    poligons_path = source + files_poligons[i]
    poligons_df = pd.read_csv( poligons_path, delimiter=';', usecols=[0,5], index_col=False )
    
    # sort with network identifier
    poligons_df.sort_values( 'network_identifier', inplace=True )
    poligons_df.reset_index( inplace=True )

    pol_net_id = poligons_df.network_identifier.values
    pol_net_id_diff_idx = np.where( np.diff( pol_net_id ) )[0]+1
    pol_net_id_diff_idx = np.insert(pol_net_id_diff_idx, 0, 0, axis=0)

    u = poligons_df.network_identifier[ pol_net_id_diff_idx ].values, 
    t = poligons_df.network_element_type[ pol_net_id_diff_idx ].values
    
    #u, c = np.unique( poligons_df.network_identifier.values, return_counts=True )
    #unique_all.append(u)
    towers_types.append( np.vstack( (u, t) ).T )

100%|██████████| 365/365 [50:23<00:00,  8.28s/it]


In [10]:
towers_types_numpy = np.concatenate(towers_types) #[ np.argsort( np.concatenate( towers_types )[:,0] ) ]
towers_types_numpy.shape, towers_types_numpy[:3]

((15415802, 2), array([[71892, 'B'],
        [72307, 'B'],
        [72922, 'B']], dtype=object))

In [11]:
sort_idx = np.argsort( towers_types_numpy[:,0] )
towers_types_sorted = towers_types_numpy[sort_idx]
towers_types_sorted

array([[71892, 'B'],
       [71892, 'B'],
       [71892, 'A'],
       ...,
       [553841790, 'B'],
       [553841790, 'B'],
       [553841790, 'B']], dtype=object)

In [47]:
towers_types_diff_idx = np.where( np.diff( towers_types_sorted[:,0] ) )[0]+1
# insert first element (zero) ## otherwise left out!
towers_types_idx = np.insert( towers_types_diff_idx, 0, 0, axis=0 )
# insert last element (size of array) ## otherwise left out!
towers_types_idx = np.append( towers_types_idx, towers_types_sorted.shape[0] )
towers_types_idx = np.vstack( (towers_types_idx[:-1], towers_types_idx[1:]) ).T
towers_types_idx.shape

(43384, 2)

In [97]:
type_to_num = dict( zip( ['A', 'B', 'C', 'D', 'E', 'U'], range(1,7) ) )
type_to_num

{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'U': 6}

In [98]:
type_to_num[ u[0] ]

2

In [102]:
towers_types_final = np.zeros( (towers_types_idx.shape[0], len(type_to_num)+1 ), dtype=np.int32 )
for n in range( towers_types_idx.shape[0] ):
    current = towers_types_sorted[ towers_types_idx[n,0]:towers_types_idx[n,1] ] 
    u, c = np.unique( current[:,1], return_counts=True )
    towers_types_final[n, 0] = current[0,0]
    for q in range(u.shape[0]):
        towers_types_final[n, type_to_num[ u[q] ] ] = c[q]

In [103]:
towers_types_final[:100]

array([[  71892,     218,     147,       0,       0,       0,       0],
       [  72307,     218,     147,       0,       0,       0,       0],
       [  72922,     218,     147,       0,       0,       0,       0],
       [  73325,     218,       0,     147,       0,       0,       0],
       [ 140903,     218,     147,       0,       0,       0,       0],
       [ 146953,     218,     147,       0,       0,       0,       0],
       [ 201108,       0,     218,       0,     147,       0,       0],
       [ 201256,       0,     218,       0,     147,       0,       0],
       [ 201465,       0,     218,       0,     147,       0,       0],
       [ 206489,       0,     218,       0,     147,       0,       0],
       [ 206823,       0,     218,       0,     147,       0,       0],
       [ 206923,       0,     218,       0,     147,       0,       0],
       [ 228930,       0,     218,       0,     147,       0,       0],
       [ 229021,       0,     218,       0,     147,       0,   

In [104]:
np.save( "unique-tower-id_all", towers_types_final )

In [105]:
temp = np.load( 'unique-tower-id_all.npy', allow_pickle=True )

In [106]:
temp

array([[    71892,       218,       147, ...,         0,         0,
                0],
       [    72307,       218,       147, ...,         0,         0,
                0],
       [    72922,       218,       147, ...,         0,         0,
                0],
       ...,
       [553841622,         0,       218, ...,       147,         0,
                0],
       [553841698,         0,       218, ...,       147,         0,
                0],
       [553841790,         0,       218, ...,       147,         0,
                0]], dtype=int32)

In [59]:
np.unique( temp[:,1], return_counts=True )

(array(['A', 'B', 'C', 'D', 'E', 'U'], dtype=object),
 array([ 8627, 15186, 12030,  7155,   384,     2]))

In [37]:
towers_types_idx.shape

(43384,)

In [42]:
for k in range( towers_types_idx.shape[0]):
    print( np.unique( towers_types_sorted[ towers_types_idx[k]:towers_types_idx[k+1] ][:,1] ) )

['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'B']
['A' 'B']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['A' 'B']
['A' 'B']
['B' 'D']
['B' 'D']
['B' 'D']
['C']
['C']
['C']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['A' 'B']
['A' 'B']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['A' 'B']
['A' 'B']
['A' 'B']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['A' 'B']
['A' 'B']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['A' 'B']
['A' 'B']
['A' 'B']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['C']
['C']
['B' 'D']
['C']
['C']
['C']


['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']


['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['B']
['B']
['B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['

['A' 'B']
['A' 'B']
['A' 'B']
['C']
['C']
['C']
['A' 'C']
['C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' '

['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['C']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A'

...

['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['B' 'D']
['B' 'D']
['B' 'D']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['A' 'B']
['A' 'B']
['A' 'B']
['C']
['B']
['C']
['B']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['C']
['C']
['C']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['A' 'C']
['

['C']
['C']
['C']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['C']
['C']
['C']
['C']
['C' 'E']
['C' 'E']
['C']
['C']
['C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['B']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['B']
['B']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['A' 'C']
['C']
['C']
['C']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['A' 'B']
['A' 'B']
['A' 'C']
['A' 'C']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C']
['C']
['C']
['C' 'E']
['C' 'E']
['C' 'E']
['B' 'D']
['B' 'D']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['B']
['B']
['B']
['B']
['B']
['A' 'B']
['A' 'B']
['A' 'B']
['C' 'E']
['C']
['C' 'E']
['C']
['C' 'E']
['C']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C' 'E']
['C'

['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['D']
['D']
['D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['C']
['B' 'D']
['B' 'D']
['C']
['C' 'E']
['A' 'C']
['A' 'C']
['A' 'C']
['A' 'C']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']
['B']


['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['A' 'B']
['A' 'B']
['B' 'D']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['A' 'B']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']
['B' 'D']


IndexError: index 43384 is out of bounds for axis 0 with size 43384